In [0]:
icd_df = spark.read.parquet("/mnt/bronze/icd_codes")
icd_df.createOrReplaceTempView("icd")

In [0]:
display(icd_df)

In [0]:
%sql
create table if not exists silver.icd( 
  icd_code string, 
  icd_code_type string, 
  code_description string, 
  inserted_date date, 
  updated_date date, 
  is_current_flag boolean
)

In [0]:
%sql 
merge into silver.icd as target
using icd as source
on target.icd_code = source.icd_code
and target.is_current_flag = true
when matched 
and ( 
  target.icd_code_type <> source.icd_code_type 
  or target.code_description <> source.code_description 
  or target.inserted_date <> source.inserted_date 
  or target.updated_date <> source.updated_date 
)  then
update set
target.updated_date = current_timestamp(),
target.is_current_flag = false

when not matched then
insert(
  icd_code,
  icd_code_type,
  code_description,
  inserted_date,
  updated_date,
  is_current_flag
) values (
  source.icd_code,
  source.icd_code_type,
  source.code_description,
  source.inserted_date,
  source.updated_date,
  source.is_current_flag
)

In [0]:
%sql
select * from silver.icd